# Модуль 2

В данной сессии выбранную модель предстоит обучить и протестировать её работу

---

In [3]:
# Импортирование необходимых библиотек, по ходу будут добавлятся другие
import pandas as pd
import numpy as np

In [4]:
# Импорт данных
df=pd.read_csv("data/dataset_clusterd.csv", sep="\t", encoding="utf-16")


In [5]:
df['значимость_раздела_wsss'] = pd.to_numeric(df['значимость_раздела_wsss'],errors='coerce')
df["значимость_раздела_wsss"]=df["значимость_раздела_wsss"].fillna(float(int(df["значимость_раздела_wsss"].mean())))
df.drop(df.iloc[:,8:],inplace=True,axis=1)
df.head()

,название_компетенции_wsss,описание_компетенции_wsss,название_раздела_wsss,значимость_раздела_wsss,признаки_раздела_wsss,cluster,nameed_clusters
0,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,обработка поверхности,15.0,специалист должен знать понимать специализиров...,1,Строительство и строительные технологии
1,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,изготовление сложных элементов ювелирных издел...,20.0,специалист должен знать понимать способы испол...,1,Строительство и строительные технологии
2,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,изготовление простых элементов ювелирных изделий,20.0,специалист должен знать понимать способы испол...,1,Строительство и строительные технологии
3,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,подготовка драгоценных сплавов изготовления эл...,10.0,специалист должен знать понимать свойства спос...,1,Строительство и строительные технологии
4,Ювелирное дело,профессия ювелира прежде связана ручным трудом...,изготовление сплавов драгоценных металлов,5.0,специалист должен знать понимать состав сплаво...,1,Строительство и строительные технологии


Для того, чтобы можно было использовать все признаки, воспользуемся трансформаторами BaseEstimator, TransformerMixin и конвеером Pipeline. Также рандомизируем датасет для классификации, чтобы было более равномерное распределение данных на обучающей и тестовой выборке

In [6]:
# Рандомизация данных
df=df.sample(n=len(df))

In [7]:
# Трансформаторы для получения данных
from sklearn.base import BaseEstimator, TransformerMixin
# Конвеер для объединения шагов выполнения классификации
from sklearn.pipeline import Pipeline
# Функция стандартизации числовых значений
from sklearn.preprocessing import StandardScaler
# Функция преобразования текстовых данных под модель
from sklearn.feature_extraction.text import TfidfVectorizer

# Класс необходимый для получения и работы с текстовыми признаками
class TextSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.key]
    
# Класс необходимый для получения работы с числовыми признаками
class NumericalSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[[self.key]]

In [8]:
# Стеммер, работающий с русским языком
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian") 

# Функция стеммизации данных для образования однокоренных слов
def stemming_tokenizer(text):
    text = re.split('\W+', text)
    text = [stemmer.stem(word) for word in text]
    return text

In [9]:
# Получение стоп-слов из библиотеки nltk(и, за, а, на, как и.т.) 
from nltk.corpus import stopwords
import re
stop_words = stopwords.words("russian")

In [10]:
df.head()

,название_компетенции_wsss,описание_компетенции_wsss,название_раздела_wsss,значимость_раздела_wsss,признаки_раздела_wsss,cluster,nameed_clusters
647,Геодезия(16-22),геодезисты это специалисты которые обладают пр...,навыки общения,3.0,специалист должен знать понимать кодекс этики ...,1,Строительство и строительные технологии
572,Интернет вещей,интернет вещей internet of things iot это конц...,разработка интерфейса мониторинга управления в...,15.0,специалист должен знать понимать принципы прое...,2,Сфера услуг
153,Сантехника и отопление (Юниоры),специалист сантехнике отоплению работает комме...,устанавливать кронштейны изготавливать сборочн...,40.0,специалист должен знать понимать способы приме...,2,Сфера услуг
212,Предпринимательство (Юниоры),это командные соревнования каждой команде учас...,бизнес процесс,16.0,специалист должен знать понимать суть бизнес п...,1,Строительство и строительные технологии
207,Производство мебели,компетенция производство мебели включает изгот...,навыки коммуникации межличностного общения,5.0,специалист должен знать понимать важность форм...,1,Строительство и строительные технологии


## Обучение
Для обучения необходимы пробразованные признаки для классификации. Построим конвейер для дальнейшего обучения моделей.

In [11]:
# Подгонка признаков под конвейер для корректной работы с моделью
dictionary = Pipeline([
    ('selector', TextSelector(key="описание_компетенции_wsss")),
    ('tfidf', TfidfVectorizer(stop_words = stop_words,tokenizer=stemming_tokenizer))
    ])

dictionary.fit_transform(df).todense()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['бол', 'больш', 'будт', 'быт', 'вед', 'впроч', 'всег', 'всегд', 'даж', 'друг', 'е', 'ег', 'ем', 'есл', 'ест', 'ещ', 'зач', 'зде', 'ил', 'иногд', 'когд', 'конечн', 'куд', 'лучш', 'межд', 'мен', 'мног', 'мо', 'можн', 'нег', 'нельз', 'нибуд', 'никогд', 'нич', 'опя', 'посл', 'пот', 'почт', 'разв', 'сво', 'себ', 'совс', 'теб', 'тепер', 'тог', 'тогд', 'тож', 'тольк', 'хорош', 'хот', 'чег', 'чут', 'эт'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


matrix([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.05450137, ..., 0.        , 0.04104278,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

In [12]:
# Подгонка признаков под конвейер для корректной работы с моделью
title = Pipeline([
    ('selector', TextSelector(key="название_раздела_wsss")),
    ('tfidf', TfidfVectorizer(stop_words = stop_words,tokenizer=stemming_tokenizer))
    ])

title.fit_transform(df).todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [13]:
# Подгонка признаков под конвейер для корректной работы с моделью
ves_title = Pipeline([
    ('selector', NumericalSelector(key="значимость_раздела_wsss")),
    ('tfidf', StandardScaler())
    ])

ves_title.fit_transform(df)

array([[-1.15501713],
       [-0.05145834],
       [ 2.2476225 ],
       [ 0.0405049 ],
       [-0.97109067],
       [ 0.0405049 ],
       [ 0.68424753],
       [ 0.40835783],
       [-0.97109067],
       [-0.5112745 ],
       [ 1.32799016],
       [-0.97109067],
       [-0.97109067],
       [ 0.40835783],
       [ 2.2476225 ],
       [ 0.868174  ],
       [ 0.0405049 ],
       [-0.2353848 ],
       [-0.97109067],
       [-0.5112745 ],
       [-0.97109067],
       [-0.5112745 ],
       [-0.97109067],
       [ 0.0405049 ],
       [-0.5112745 ],
       [-0.5112745 ],
       [-0.14342157],
       [-0.5112745 ],
       [ 0.40835783],
       [ 0.868174  ],
       [-0.5112745 ],
       [-0.05145834],
       [ 0.22443136],
       [-0.05145834],
       [ 2.15565926],
       [ 2.33958573],
       [ 0.22443136],
       [ 3.16725483],
       [ 0.77621076],
       [-0.32734803],
       [-1.15501713],
       [ 0.40835783],
       [-0.5112745 ],
       [-0.97109067],
       [ 0.0405049 ],
       [-0

In [14]:
# Подгонка признаков под конвеер для корректной работы с моделью
title_feat = Pipeline([
    ('selector', TextSelector(key="признаки_раздела_wsss")),
    ('tfidf', TfidfVectorizer(stop_words = stop_words,tokenizer=stemming_tokenizer))
    ])

title_feat.fit_transform(df).todense()

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['бол', 'больш', 'будт', 'быт', 'вед', 'впроч', 'всег', 'всегд', 'даж', 'друг', 'е', 'ег', 'ем', 'есл', 'ест', 'ещ', 'зач', 'зде', 'ил', 'иногд', 'когд', 'конечн', 'куд', 'лучш', 'межд', 'мен', 'мног', 'мо', 'можн', 'нег', 'нельз', 'нибуд', 'никогд', 'нич', 'опя', 'посл', 'пот', 'почт', 'разв', 'сво', 'себ', 'совс', 'теб', 'тепер', 'тог', 'тогд', 'тож', 'тольк', 'хорош', 'хот', 'чег', 'чут', 'эт'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
df.head()

,название_компетенции_wsss,описание_компетенции_wsss,название_раздела_wsss,значимость_раздела_wsss,признаки_раздела_wsss,cluster,nameed_clusters
647,Геодезия(16-22),геодезисты это специалисты которые обладают пр...,навыки общения,3.0,специалист должен знать понимать кодекс этики ...,1,Строительство и строительные технологии
572,Интернет вещей,интернет вещей internet of things iot это конц...,разработка интерфейса мониторинга управления в...,15.0,специалист должен знать понимать принципы прое...,2,Сфера услуг
153,Сантехника и отопление (Юниоры),специалист сантехнике отоплению работает комме...,устанавливать кронштейны изготавливать сборочн...,40.0,специалист должен знать понимать способы приме...,2,Сфера услуг
212,Предпринимательство (Юниоры),это командные соревнования каждой команде учас...,бизнес процесс,16.0,специалист должен знать понимать суть бизнес п...,1,Строительство и строительные технологии
207,Производство мебели,компетенция производство мебели включает изгот...,навыки коммуникации межличностного общения,5.0,специалист должен знать понимать важность форм...,1,Строительство и строительные технологии


### Сборка всех признаков
Введём в FeatureUnion(Метод, объединяющий признаки) пробразованные признаки, такие как: `название_раздела_wsss`, `значимость_раздела_wsss`, `признаки_раздела_wsss`, потому что именно этих данных достаточно для модели классификации. 

In [17]:
from sklearn.pipeline import FeatureUnion
# Формирование конвейера 
feat = FeatureUnion([("title", title),
                    ("ves_title", ves_title),
                    ("title_feat", title_feat)])

# Переменная, которая пойдёт на разделение набора и последующее обучкение и тестирование
X=feat.fit_transform(df).todense()


# Формирование конвейера 


### Разделение набора данных
Поскольку в датасете содержится достаточно мало данных, то стоит их экономить. И абсолютное большинство передать на обучений. Обычно рекомендованно использовать 1/3 данных для тестирования. Но опять же данных в датасете мало, поэтому будет использована 1/10 данных для тестирования

In [19]:
from sklearn.model_selection import train_test_split
# Целевая переменная, которая содержит названия кластеров
y=df["nameed_clusters"]
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.1)

## Методы классификации
В тестировании моделей будут принимать 3 метода классификации:
### SVC(Метод опорных векторов)
Метод Опорных Векторов или SVM (от англ. Support Vector Machines) — это линейный алгоритм используемый в задачах классификации и регрессии. Данный алгоритм имеет широкое применение на практике и может решать как линейные так и нелинейные задачи. Суть работы “Машин” Опорных Векторов проста: алгоритм создает линию или гиперплоскость, которая разделяет данные на классы. Данный метод лучше всего подходит для задач классификации с текстом
### Наивный байесовский классификатор (Naive Bayes)
Такой классификатор вычисляет вероятность принадлежности объекта к какому-то классу. Эта вероятность вычисляется из шанса, что какое-то событие произойдёт, с опорой на уже на произошедшие события.
Каждый параметр классифицируемого объекта считается независимым от других параметров.
### Логистическая регрессия (Logistic Regression)
Логистическая регрессия выводит прогнозы о точках в бинарном масштабе — нулевом или единичном. Если значение чего-либо равно либо больше 0.5, то объект классифицируется в большую сторону (к единице). Если значение меньше 0.5 — в меньшую (к нулю).

У каждого признака есть своя метка, равная только 0 или только 1. Логистическая регрессия является линейным классификатором и поэтому используется, когда в данных прослеживается какая-то линейная зависимость.

---

In [20]:
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression

---

### Метрики 
В качестве метрик возьмем все то, что располагает в себе метод `classification_report`. Берутся эти метрики потому что они предоставлют достаточно информации, чтобы сравнить несколько моделей классификации и выбрать по этим метрикам самую лучшую модель. А именно:
* `Accuracy` - доля правильных ответов алгоритма
* `Precision` - можно интерпретировать как долю объектов, названных классификатором положительными и при этом действительно являющимися положительными
* `recall` - показывает, какую долю объектов положительного класса из всех объектов положительного класса нашел алгоритм
* `f-score` - эта мера может быть интерпретирована как средневзвешенное значение Precision и recall, где оценка F1 достигает своего наилучшего значения при 1 и худшего значения при 0
* `macro avg` - вычисляет метрику независимо для каждого класса, а затем принимает среднее значение
* `weighted avg` - вычисляет оценку F1 для каждого класса независимо, но когда она складывает их вместе, используется вес, который зависит от количества истинных меток каждого класса

Атрибут `support` - показывает количество образцов истинного ответа, которые лежат в этом классе.

---

In [21]:
from sklearn.metrics import classification_report

## Обучение моделей и их сравнение

In [22]:
# Метод SVC(Метод опорных векторов)
pipeline_svc=SVC()

# Обучение классификатора на обучающих данных
pipeline_svc.fit(X_train,y_train)
# Предсказание целевой переменной на тестирующих данных
preds_scv=pipeline_svc.predict(X_test)
# Вывод со всеми описанными метриками ранее
print(classification_report(y_test, preds_scv))

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


                                              precision    recall  f1-score   support

Информационные и коммуникационные технологии       0.00      0.00      0.00        12
        Производство и инженерные технологии       0.00      0.00      0.00         5
     Строительство и строительные технологии       0.73      1.00      0.84        59
                                 Сфера услуг       0.00      0.00      0.00         5

                                   micro avg       0.73      0.73      0.73        81
                                   macro avg       0.18      0.25      0.21        81
                                weighted avg       0.53      0.73      0.61        81



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Метод `SVC` показал точность(accuracy): **`0.78`**

In [23]:
# Метод GaussianNB
pipeline_gnb=GaussianNB()

# Обучение классификатора на обучающих данных
pipeline_gnb.fit(X_train,y_train)
# Предсказание целевой переменной на тестирующих данных
preds_gnb=pipeline_gnb.predict(X_test)
# Вывод со всеми описанными метриками ранее
print(classification_report(y_test, preds_gnb))

                                              precision    recall  f1-score   support

Информационные и коммуникационные технологии       0.73      0.67      0.70        12
        Производство и инженерные технологии       1.00      1.00      1.00         5
     Строительство и строительные технологии       0.90      0.93      0.92        59
                                 Сфера услуг       0.75      0.60      0.67         5

                                   micro avg       0.88      0.88      0.88        81
                                   macro avg       0.84      0.80      0.82        81
                                weighted avg       0.87      0.88      0.87        81



Метод `GaussianNB` показал точность: **`0.79`**

In [24]:
# Метод LogisticRegression
pipeline_lr=LogisticRegression()

# Обучение классификатора на обучающих данных
pipeline_lr.fit(X_train,y_train)
# Предсказание целевой переменной на тестирующих данных
preds_lr=pipeline_lr.predict(X_test)
# Вывод со всеми описанными метриками ранее
print(classification_report(y_test, preds_lr))

                                              precision    recall  f1-score   support

Информационные и коммуникационные технологии       1.00      0.67      0.80        12
        Производство и инженерные технологии       1.00      0.40      0.57         5
     Строительство и строительные технологии       0.87      1.00      0.93        59
                                 Сфера услуг       1.00      0.60      0.75         5

                                   micro avg       0.89      0.89      0.89        81
                                   macro avg       0.97      0.67      0.76        81
                                weighted avg       0.90      0.89      0.88        81



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Метод `LogisticRegression` показал точность: **`0.83`**

**Вывод по обучению:** как видим из представленных данных лучше всего себя показал классификатор `LogisticRegression` с точностью **`0.83`**

# Подготовка отчёта:
* **Показатели точности работы модели:** Во внимание можно взять абсолютно все метрики, поставляющиеся в методе `classification_report`, но за основу сравнения моделей была взята метрика accuracy, которая показывает долю правильных ответов алгоритма
* **Точность выбранной модели в сравнении с другими:** Все модели показали достаточно хороший результат на отметке не ниже **`0.78`**, но модель `LogisticRegression` показала себя лучше всего, на отметке **`0.83`**. Считаю такой результат получился, потому что этот классификатор достаточно хорошо работает с большим количеством признаков и не требует больших вычислительных мощностей из-за составления линейного алгоритма
